In [1]:
from numpy import *
from matplotlib.pyplot import *
import scipy.linalg
import matplotlib.lines as mlines
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize']= 16,5
from __future__ import division

In [2]:
import pandas as pd
import csv
import os
import time
import gc

In [104]:
def load_data():
    """
Function to load data
    :return: data vector, data target, data mean, data standar deviation 
    """
    data = np.genfromtxt('Resources/data.csv', delimiter=',')
    d_mean = np.mean(data)
    d_std = np.std(data)
    valtarget = (data[-24:-12])
    data = ((data - d_mean) / d_std)*0.1
    return data, valtarget, d_mean, d_std

In [108]:
def load_dataTest():
    """
Function to load data
    :return: data vector, data target, data mean, data standar deviation 
    """
    data = np.genfromtxt('Resources/data.csv', delimiter=',')
    d_mean = np.mean(data)
    d_std = np.std(data)
    target = data[-12:-6]
    data = ((data - d_mean) / d_std)    
    return data, target, d_mean, d_std

In [5]:
def get_TS():
    rawData = pd.read_csv('Resources/Consulta_Banxico.csv', sep=',', encoding='latin1', skiprows=15, skip_blank_lines=True)
    Data = rawData.drop(rawData.index[:2])
    Data = Data.reset_index(drop = True)
    Data.columns = ['Fecha' , 'Tasa de crecimiento Mensual']
    TS = pd.Series(Data['Tasa de crecimiento Mensual'].astype(float))
    TS.index= Data['Fecha']
    del Data
    return TS

In [6]:
def generate_win_matrix(inSize, resSize, scaleFactor=1):
    """
Function to generate W in matrix with a uniform distribution between [-1,1]
in other Function the uniform distribution can be moved to any range
    :param inSize: number of inputs without interceptor
    :param resSize: size of reservoir
    :param scaleFactor: Scale Factor for the matrix by default 1
    :return: W in Matrix
    """
    Win = (random.uniform(low=-0.5, high=0.5, size=(resSize, 1 + inSize))) * scaleFactor
    return Win

In [7]:
def generate_w_matrix(resSize):
    """
Function to generate W matrix with a uniform distribution between [-1,1]
 In other function the uniform distribution can be moved to any range
    :param resSize: size of reservoir
    :return: W matrix
    """
    W = random.uniform(low=-0.5, high= 0.5, size = (resSize,resSize))
    return  W

In [8]:
def get_spectral_radius(W):
    """
Function to get the spectral radius of W matrix
    :param W: W matrix
    :return: spectral radius
    """
    rhoW = max(abs(linalg.eig(W)[0]))
    return  rhoW

In [9]:
def scale_w(W,rhoW, SRS = 2):
    """
Function to apply spectral radius scale factor
    :param W: W matrix
    :param rhoW: spectral radius
    :param SRS: spectral radius scale factor by default 1
    :return: W
    """
    W*= (SRS/ rhoW)
    return W

In [10]:
def get_x_yt_matrix(data,inSize, resSize,trainLen,initLen):
    """
Function to get X and Yt Matrix to save activation states and prediction targets
    :type data: normalized data
    :param inSize: size of the input 
    :param resSize: size of reservoir
    :param trainLen: length of train data
    :param initLen:  length of initial data
    :return: X, Yt
    """
    X = zeros((1+inSize+resSize,trainLen-initLen))
    Yt = data[None,initLen+1:trainLen+1]
    return X,Yt

In [11]:
def get_activation_states(data,resSize,trainLen,initLen,Win,W,a,X):
    """
Function to get the activations states from the reservoir
    :param data: train data
    :param resSize: reservoir size
    :param trainLen: length of train data
    :param initLen: inital data to be ignored
    :param Win: W in Matrix
    :param W: W matrix
    :param a: leaking rate
    :param X: matrix of Activations per input
    :return: last activation vector, matrix of activations X
    """
    x = zeros((resSize,1))
    for t in range(trainLen):
        u = data[t]
        x_u = tanh( dot( Win, vstack((1,u)) ) + dot( W, x ) )
        #x_u = sigmoid( dot( Win, vstack((1,u)) ) + dot( W, x ) )
        x = (1-a)*x + a*x_u 
        if t >= initLen:
            X[:,t-initLen] = vstack((1,u,x))[:,0]
    return x,X

In [12]:
def train_output_l2(inSize,resSize,X,Yt,reg = 1e-8):
    """
Function to train readout with ridge regression to address overfitting
    :param inSize: size of input
    :param resSize: reservoir size
    :param reg: penalization parameter
    :param X: Matrix of activations
    :param Yt: target values
    :return Wout: weights from reservoir and input to output layer
    """
    X_T = X.T 
    Wout = dot( dot(Yt,X_T), linalg.inv( dot(X,X_T) + reg*eye(1+inSize+resSize) ) )
    return Wout

In [13]:
def get_predictions(data,outSize,trainLen,testLen,a,Win,W,Wout,x):
    """
Function to get the predictions of the normalized data
    :param data: train data
    :param outSize: output size
    :param trainLen: length of train data
    :param testLen: length of test data
    :param a: leaking rate
    :param Win: W in Matrix
    :param W: W matrix
    :param Wout: W out Matrix
    :param x: last activations states
    :return: vector of predictions
    """
    Y = zeros((outSize,testLen))
    u = data[trainLen]
    for t in range(testLen):
        x_update = tanh( dot( Win, vstack((1,u))) + dot( W, x )) 
        #x_update = sigmoid( dot( Win, vstack((1,u))) + dot( W, x ))
        x = (1-a)*x + a*x_update
        y = dot( Wout, vstack((1,u,x)) )
        Y[:,t] = y
        # generative mode, this take the prediction and use it to make other prediction:
        u = y
        ## this would be a predictive mode, this made a prediccion and save it, then use the true value to make a new prediction:
        #u = data[trainLen+t+1]
    return Y

In [92]:
def denormalize_predictions(d_mean,d_std,Y):
    """
Function to denormalize predictions
    :param d_mean: data mean
    :param d_std: data standard deviation
    :param Y: predictions
    :return: denormalized predictions
    """
    Y1 = ((Y * d_std) + d_mean)/0.1
    return Y1

In [92]:
def denormalize_predictions(d_mean,d_std,Y):
    """
Function to denormalize predictions
    :param d_mean: data mean
    :param d_std: data standard deviation
    :param Y: predictions
    :return: denormalized predictions
    """
    Y1 = ((Y * d_std) + d_mean)*0.1
    return Y1

In [15]:
def get_MSE(predictions, target):
    """
    Function to get MSE from diference between predictions and real value (ouptuts)
    :param predictions: predictions made by ffnn
    :param target: true values from output values
    :return: MSE
    """
    return  (np.sum((target - predictions)**2))/len(predictions)

In [16]:
def SolutionValidation(Predictions,MSE,bestESN,bestMSE,bestPredictions,currentSetup):
    """
Function to validate ESN model
    :param Predictions: Prediction of current ESN
    :param MSE: MSE of current ESN
    :param bestESN: best setup of ESN so far
    :param bestMSE: MSE of best ESN so far
    :param bestPredictions: predictions of best ESN so far
    :param currentSetup: current ESN setup
    :return: bestESN, bestPredictions, bestMSE
    """
    if((float)(MSE) < (bestMSE)):
        bestESN = currentSetup
        bestMSE = MSE
        bestPredictions = Predictions
    return bestESN,bestPredictions,bestMSE

In [17]:
def SolutionValidationTest(Predictions,MSE,bestESN,bestMSE,bestPredictions,currentSetup,Matrix,bMatrix):
    """
Function to validate ESN model
    :param Predictions: Prediction of current ESN
    :param MSE: MSE of current ESN
    :param bestESN: best setup of ESN so far
    :param bestMSE: MSE of best ESN so far
    :param bestPredictions: predictions of best ESN so far
    :param currentSetup: current ESN setup
    :return: bestESN, bestPredictions, bestMSE
    """
    if((float)(MSE) < (bestMSE)):
        bestESN = currentSetup
        bestMSE = MSE
        bestPredictions = Predictions
        bMatrix = Matrix
    return bestESN,bestPredictions,bestMSE,bMatrix

In [18]:
def init_best(numl2):
    best_setup = (0,0,0,0,0)
    best_mse = 100000
    best_pred = (0,0,0,0,0,0)
    return best_setup,best_mse,best_pred

In [19]:
def toCSVandPlot(folder,name,esn,mse,pred,TS):
    if not os.path.isdir(folder):
        os.makedirs(folder)
    with open(folder + name + '.csv','w+') as csvfile:
        spamwriter= csv.writer(csvfile,delimiter=',')
        spamwriter.writerow(['ESN configuration: ',esn])
        spamwriter.writerow(['predictions'])
        spamwriter.writerow(pred)
        spamwriter.writerow(['MSE:' , mse])
        csvfile.close()        
    #plot_Solutions(folder,name,pred,TS)
    gc.collect()

In [20]:
def toCSV(folder,name,b_esn,mse,pred,c_esn,list,TS):
    if not os.path.isdir(folder):
        os.makedirs(folder) 
    with open(folder + name + '.csv','w+') as csvfile:
        spamwriter= csv.writer(csvfile,delimiter=',')
        spamwriter.writerow(['ESN configuration: ',b_esn])
        spamwriter.writerow(['predictions'])
        spamwriter.writerow(pred)
        spamwriter.writerow(['MSE:' , mse])
        spamwriter.writerow(['Current Setup w_o l2 penalty',c_esn])
        spamwriter.writerow(['Predictions'])
        spamwriter.writerows(list)
        csvfile.close()        
    #plot_Solutions2(folder,name,pred,list,TS)
    gc.collect()

In [90]:
def writeIterations(listPredTuple):
    folder = "Results/iteraciones"  +"/" 
    with open(folder + 'Validation_predicions_2015_1' + '.csv','w+') as csvfile:
        spamwriter= csv.writer(csvfile,delimiter=',')
        spamwriter.writerow(['Param1','Param2','Param3','Param4','Param5','Pred1','Pred2','Pred3','Pred4','Pred5','Pred6',
                             'Pred7','Pred8','Pred9','Pred10','Pred11','Pred12','MSE','Time'])
        for x in listPredTuple:
                    y = np.append(np.append(x[0],x[2]),np.append(x[1],x[3]))
                    spamwriter.writerow(y)
    csvfile.close()

In [91]:
def writePredictions(listPredTuple):
    folder = "Results/iteraciones"  +"/" 
    with open(folder + 'Test_predicions_2015_1' + '.csv','w+') as csvfile:
        spamwriter= csv.writer(csvfile,delimiter=',')
        spamwriter.writerow(['Param1','Param2','Param3','Param4','Param5','Pred1','Pred2','Pred3','Pred4','Pred5','Pred6','MSE'])
        for x in listPredTuple:                
                    y = np.append(x[0],x[1])
                    spamwriter.writerow(y)
    csvfile.close()

In [23]:
def plot_Solutions(folder,name,Pred,TS):
    """
    Function to Plot List of predictions, mean predictions and Original Time Serie
    :param folder: path of file
    :param name: name of file
    :param Pred: predictions
    :param TS: Original Time Serie
    :return: matplotlib fig
    """
    fig = figure()
    TSValOutputs = TS[len(TS)-7:len(TS)].copy()
    TSValOutputs.plot(style= 'ko-')
    
    TSPred = pd.Series(Pred.astype(float), index = TS.index[len(TS)-12:len(TS)-6])
    TSPrediction = TS[len(TS)-7:len(TS)].copy()
    TSPrediction[1:] = TSPred
    TSPrediction.plot(style= 'bo-')
    lineMean = mlines.Line2D([],[],color='blue', marker='o',label = 'Prediction')
    lineVal = mlines.Line2D([],[],color='black', marker='o',label = 'True Values')
    legend(handles = [lineMean,lineVal],loc= 1)
    fig.savefig(folder + name + ".png")
    fig.clf()
    del fig
    gc.collect()
    return None

In [24]:
def plot_Solutions2(folder,name,Pred,list,TS):
    """
    Function to Plot List of predictions, mean predictions and Original Time Serie
    :param folder: path of file
    :param name: name of file
    :param Pred: predictions
    :param TS: Original Time Serie
    :return: matplotlib fig
    """
    fig = figure()
    TSValOutputs = TS[len(TS)-7:len(TS)].copy()
    TSValOutputs.plot(style= 'ko-')
    
    TSPred = pd.Series(Pred.astype(float), index = TS.index[len(TS)-12:len(TS)-6])
    TSPrediction = TS[len(TS)-7:len(TS)].copy()
    TSPrediction[1:] = TSPred
    TSPrediction.plot(style= 'bo-')
    lineMean = mlines.Line2D([],[],color='blue', marker='o',label = 'Prediction')
    lineVal = mlines.Line2D([],[],color='black', marker='o',label = 'True Values')
    legend(handles = [lineMean,lineVal],loc= 1)
    fig.savefig(folder + name + ".png")
    fig.clf()
    del fig
    gc.collect()
    return None

In [25]:
def get_denormilze_prediction(data,d_mean,d_std,outSize,trainLen,testLen,a,Win,W,Wout,x):
    Y = get_predictions(data,outSize,trainLen,testLen,a,Win,W,Wout,x)
    Y = denormalize_predictions(d_mean,d_std,Y)
    return Y

In [26]:
def get_best_setup(ParameterList):
    """
Function to test several ESN model for the best setup
    :param ParameterList: List of vectors for different values of the parameters in this order:
    resSize, inputScale, SpectralRadiusScale, leakingRate, l2penalty
    """    
    start = time.time()
    random.seed()
    data,target,d_mean,d_std = load_data()
    trainLen =len(data) - 6
    testLen = 6
    initLen = 12
    inSize = outSize = 1
    TS = get_TS()
    time_stamp = time.strftime("%d-%m") + "_" + time.strftime("%H-%M")
    folder = "Results/iteraciones"  +"/" 
    best_setup,best_MSE,best_predictions,listl2_pen = init_best(len(ParameterList[4]))
    param1 = 0   
    for resSize in ParameterList[0]:
       # name1 = ((str)(param1) + '-')
        param1 += 1
        param2 = 0
        for WinScale in ParameterList[1]:
            #name2 = name1 + (str)(param2) + '-'
            param2 += 1
            param3 = 0
            Win = generate_win_matrix(inSize,resSize,WinScale)
            for SRS in ParameterList[2]:
                #name3 = name2 + (str)(param3) + '-'
                param3 += 1
                param4 = 0
                W = generate_w_matrix(resSize)            
                rhoW = get_spectral_radius(W)
                W = scale_w(W,rhoW,SRS)
                X,Yt = get_x_yt_matrix(data,inSize,resSize,trainLen,initLen)
                for a in ParameterList[3]:
                    #name4 = name3 + (str)(param4) + '-'
                    param4 += 1
                    param5 = 0
                    x,X = get_activation_states(data,resSize,trainLen,initLen,Win,W,a,X)
                    for reg in ParameterList[4]:
                        #name5 = name4 + (str)(param5)
                        param5 += 1
                        Wout = train_output_l2(inSize,resSize,X,Yt,reg)
                        Y = get_predictions(data,outSize,trainLen,testLen,a,Win,W,Wout,x)
                        Y = denormalize_predictions(d_mean,d_std,Y)
                        MSE = get_MSE(Y[0],target)
                        listl2_pen[param5-1]= (np.append(Y,MSE))
                        #add list of results for l2 penalty
                        currentSetup = (resSize,WinScale,SRS,a,reg)
                        gc.collect()
                        best_setup,best_predictions,best_MSE = SolutionValidation(Y[0],MSE,best_setup,best_MSE,best_predictions,currentSetup)
                  # toCSV(folder,name5,best_setup,best_MSE,best_predictions,currentSetup[:-1],listl2_pen,TS)
    stop = time.time()
    duration = stop - start
    toCSVandPlot(folder,('best-' + time_stamp) ,best_setup,best_MSE,best_predictions,TS)
    windows_explorer_path =  'C:\Users\sa-al\PycharmProjects\AnnEchoState\Results' 
    #os.startfile(windows_explorer_path)    
    return best_setup,best_MSE,best_predictions,duration

In [27]:
def get_best_ESN(ParameterList):
    """
Function to test several ESN model for the best setup
    :param ParameterList: List of vectors for different values of the parameters in this order:
    resSize, inputScale, SpectralRadiusScale, leakingRate, l2penalty
    """
    start = time.time()
    random.seed()
    data, target, d_mean, d_std = load_data()

    # for make predictions to uknown data we have to fully train the model, for that we will use all the data except the last, because it will 
    # be use as a target in the training, and we will use a second trainLen to choose the model
    #FulltrainLen = len(data) - 6
    trainLen = len(data) - 18

    # for choose the best model we have to evaluate in a certain number of predictios with know real values
    testLen = 12

    initLen = 12
    inSize = outSize = 1
    TS = get_TS()
    time_stamp = time.strftime("%d-%m") + "_" + time.strftime("%H-%M")
    folder = "Results/iteraciones" + "/"
    best_setup, best_MSE, best_predictions = init_best(len(ParameterList[4]))
    bWin = 0
    bW = 0
    best_matrix = 0
    
    #param1 = 0
    for resSize in ParameterList[0]:
        # name1 = ((str)(param1) + '-')
        #param1 += 1
        #param2 = 0
        
        X, Yt = get_x_yt_matrix(data, inSize, resSize, trainLen, initLen)
        
        for WinScale in ParameterList[1]:
            # name2 = name1 + (str)(param2) + '-'
            #param2 += 1
            #param3 = 0
            Win = generate_win_matrix(inSize, resSize, WinScale)
            for SRS in ParameterList[2]:
                # name3 = name2 + (str)(param3) + '-'
                #param3 += 1
                #param4 = 0
                W = generate_w_matrix(resSize)
                rhoW = get_spectral_radius(W)
                W = scale_w(W, rhoW, SRS)

                # this will create the state matrix of 167 rows and a Yt vector with the last value
                # X,Yt = get_x_yt_matrix(data,inSize,resSize,FulltrainLen,initLen)
                # we create a X state matrix and Yt for training data
                # X_train,Yt_train = get_x_yt_matrix(data,inSize,resSize,trainLen,initLen)

                for a in ParameterList[3]:
                    # name4 = name3 + (str)(param4) + '-'
                    #param4 += 1
                    #param5 = 0

                    # this will fill the state matrix with all the data   
                    x, X = get_activation_states(data, resSize, trainLen, initLen, Win, W, a, X)

                    # using train and validation data to select best model
                    for reg in ParameterList[4]:
                        # name5 = name4 + (str)(param5)
                        #param5 += 1

                        # Train the output with the data to -6 observation in order to select the best model to make predictions
                        Wout = train_output_l2(inSize, resSize, X, Yt, reg)
                        # we get the predictions with training data
                        Y = get_predictions(data, outSize, trainLen, testLen, a, Win, W, Wout, x)

                        Y = denormalize_predictions(d_mean, d_std, Y)
                        MSE = get_MSE(Y[0], target)
                        # ESN = [a,Win,W,Wout,x_train]
                        currentSetup = (resSize, WinScale, SRS, a, reg)
                        gc.collect()

                        Matrix = (Win,W,Wout)
                        best_setup, best_predictions, best_MSE,best_matrix = SolutionValidationTest(Y[0], MSE, best_setup, best_MSE,
                                                                                    best_predictions, currentSetup,Matrix,best_matrix)                       

    # Once we have the best setup we use the test values
    # We retrain de model with all the data, but for the train the last row of the state matrix shouldn't be used, this due we don't have
    # the target value for this row (this is the last observation, it will be use for the prediction

    #Wout = train_output_l2(inSize, best_setup[0], bX, Yt, best_setup[4])
    # with the model re trained we made the predictions for test data
    #Ytest = get_predictions(data, outSize, trainLen, testLen, best_setup[3], bWin, bW, Wout, bx)
    #Ytest = denormalize_predictions(d_mean, d_std, Ytest)

    stop = time.time()
    duration = stop - start
    # toCSVandPlot(folder,('best-' + time_stamp) ,best_setup,best_MSE,best_predictions,TS)
    windows_explorer_path = 'C:\Users\sa-al\PycharmProjects\AnnEchoState\Results'
    # os.startfile(windows_explorer_path)    
    return best_setup, best_MSE, best_predictions, duration,best_matrix
    # return ESN,data

In [68]:
def TestESN(ValESN):
    """
    Function to test the best val ESN to find best ESN model for the best setup
    :param ParameterList: List of vectors for different values of the parameters in this order:
    resSize, inputScale, SpectralRadiusScale, leakingRate, l2penalty
    """
    random.seed()
    predTest = list()
    data, target, d_mean, d_std = load_dataTest()

    # for make predictions to uknown data we have to fully train the model, for that we will use all the data except the last, because it will 
    # be use as a target in the training, and we will use a second trainLen to choose the model
    #FulltrainLen = len(data) - 6
    trainLen = len(data) - 13

    # for choose the best model we have to evaluate in a certain number of predictios with know real values
    testLen = 6

    initLen = 12
    inSize = outSize = 1
    #TS = get_TS()

    for tuple in ValESN:
        setup = tuple[0]
        resSize = setup[0]
        WinScale = setup[1]
        SRS = setup[2]
        a = setup[3]
        reg = setup[4]
        
        X, Yt = get_x_yt_matrix(data, inSize, resSize, trainLen, initLen)

        #Win = generate_win_matrix(inSize, resSize, WinScale)
        Win = tuple[4][0]

        #W = generate_w_matrix(resSize)
        #rhoW = get_spectral_radius(W)
        #W = scale_w(W, rhoW, SRS)
        W = tuple[4][1]
        x, X = get_activation_states(data, resSize, trainLen, initLen, Win, W, a,
                                     X)  # Train the output with the data to -6 observation in order to select the best model to make predictions
        Wout = train_output_l2(inSize, resSize, X, Yt, reg)
        # we get the predictions with training data
        Y = get_predictions(data, outSize, trainLen, testLen, a, Win, W, Wout, x)

        Y = denormalize_predictions(d_mean, d_std, Y)
        result =(setup,Y)
        predTest.append(result)
        gc.collect()  # Once we have the test we clean the memory
    writePredictions(predTest)

In [28]:
#ParameterList_0_3 = [[100,200,300,400,500],[1,1.2,1.5,1.8,2,2.5,3,3.5,4],[0.5,1,1.2,1.5,2,2.5,3,3.5],[0.1,0.2,0.3,0.4,0.5,0.6,0.7],[1e-13,1e-12,1e-11,1e-10,1e-9,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1]]

In [29]:
ParameterList = [[100,200,300,400],[0.5,1.2,1.5,1.8,2,2.5],[0.3,0.5,0.8,1,1.5],[0.4,0.5,0.6,0.7],[1e-12,1e-11,1e-10,1e-9,1e-8,1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1e-0]]

In [30]:
listPredTuple = list()
#for x in range(5):
 #  bestPredtuple = get_best_setup(ParameterList)   
  # listPredTuple.append(bestPredtuple)
#writeIterations(listPredTuple)

Aprox 1.- probar datos en mejor red

In [31]:
#ParameterList = [[100,200],[-0.5,-0.1,0.1,0.3,0.5,0.7,1,1.3],[0.3,0.4,0.5,0.6,0.8,1,1.2],[0.2,0.3,0.4,0.5,0.6,0.7,0.8],[1e-9,1e-8,1e-7,1e-6,1e-5,1e-4,1e-3]]

In [103]:
ParameterList = [[100],[0.1,0.3,0.5,0.7,1,1.3,1.5],[0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1],[0.2,0.3,0.4,0.5,0.6,0.7,0.8],[1e-9,1e-8,1e-7]]

In [100]:
#ParameterList = [[100,200],[-0.3,0.1,0.5,0.7,1,1.3],[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1],[0.2,0.3,0.4,0.5,0.6,0.7,0.8],[1e-7,1e-6,1e-5,1e-4,1e-3,1e-2]]

In [106]:
predVal = list()
for x in range(2):
   bestPredtuple = get_best_ESN(ParameterList)  
   predVal.append(bestPredtuple)
writeIterations(predVal)

In [107]:
TestESN(predVal)

In [72]:
predTest = list()
data, target, d_mean, d_std = load_dataTest()
random.seed()
#for make predictions to uknown data we have to fully train the model, for that we will use all the data except the last, because it will 
#be use as a target in the training, and we will use a second trainLen to choose the model
trainLen = len(data) - 13

#for choose the best model we have to evaluate in a certain number of predictios with know real values
for solution in predVal:
    setup = solution[0]    
    testLen = 6
    
    initLen = 12
    inSize = outSize = 1
    #for solution in predVal:
    #setup = solution[0]
    resSize = setup[0]
    WinScale = setup[1]
    #Win = generate_win_matrix(inSize, resSize, WinScale)
    Win = solution[4][0]
    
    SRS = setup[2]
    
    #W = generate_w_matrix(resSize)
    #rhoW = get_spectral_radius(W)
    #W = scale_w(W, rhoW, SRS)
    W = solution[4][1]
    
    #this will create the state matrix of 167 rows and a Yt vector with the last value
    X, Yt = get_x_yt_matrix(data, inSize, resSize, trainLen, initLen)
    
    a = setup[3]
    
    #this will fill the state matrix with all the data   
    x, X = get_activation_states(data, resSize, trainLen, initLen, Win, W, a, X)
    
    #using train and validation data to select best model
    reg = setup[4]  #Train the output with the data to -6 observation in order to select the best model to make predictions
    Wout = train_output_l2(inSize, resSize, X, Yt,reg)
    #Wout = solution[4][2]
    #we get the predictions with training data
    Y2 = get_predictions(data, outSize, trainLen, testLen, a, Win, W, Wout, x)
    Y2 = denormalize_predictions(d_mean, d_std, Y2)
    MSE = get_MSE(Y2[0], target)
    test = (solution[0], Y2[0], MSE)
    predTest.append(test)
gc.collect()

7

In [73]:
writePredictions(predTest)

In [82]:
predVal[0][4][2]

array([[ -6.59909541e+01,  -5.51477726e-02,  -8.04715903e+02,
         -5.16212266e+02,   4.39658302e+02,   3.63337031e+02,
         -1.56407383e+02,  -7.58262078e+02,  -1.43071102e+02,
         -2.80058454e+02,   4.53700659e+02,   8.29753579e+02,
          2.31414657e+02,   1.17563935e+03,  -6.94295705e+02,
          8.95218314e+02,  -7.96688247e+02,   2.06320204e+02,
          1.10152751e+03,  -4.15291842e+02,   5.87267881e+01,
          1.80830612e+02,  -5.52000387e+02,   9.38830749e+02,
         -5.13772156e+02,  -5.75654474e+02,   5.34172081e+01,
          1.33054476e+03,   2.17064294e+02,  -4.15502692e+02,
         -3.31604130e+02,  -1.50598027e+03,  -7.44006989e+02,
          1.12741270e+03,   3.28479355e+02,   1.61611977e+02,
         -7.77953739e+02,   5.17767218e+02,  -5.62334468e+02,
         -2.60773658e+02,   6.01286758e+02,   2.54848638e+02,
          1.13610064e+03,   1.92215023e+03,   1.91512047e+02,
         -1.07173019e+03,  -2.67363963e+02,   2.92052605e+02,
        